In [76]:
from dotenv import load_dotenv
from langchain_groq import ChatGroq
from langchain_core.tools import tool
from langgraph.prebuilt import ToolNode
import os
from langgraph.graph import END , MessageGraph
from typing import Literal, List
from langchain_core.messages import BaseMessage, HumanMessage
load_dotenv()

True

In [77]:
os.environ['LANGCHAIN_TRACING_V2'] = os.getenv('LANGCHAIN_TRACING_V2')
os.environ['LANGCHAIN_ENDPOINT'] = os.getenv('LANGCHAIN_ENDPOINT')
os.environ['LANGCHAIN_API_KEY'] = os.getenv('LANGCHAIN_API_KEY')
os.environ['LANGCHAIN_PROJECT'] = os.getenv('LANGCHAIN_PROJECT')
os.environ['GROQ_API_KEY'] = os.getenv('GROQ_API_KEY')

In [78]:
@tool
def multiply(first_number: int, second_number: int):
    """Multiplica dos números entre sí."""
    return first_number * second_number

### Configuración del modelo LLM:

In [79]:
GROQ_LLM = ChatGroq(temperature=0,model="llama3-70b-8192")

### Añadir herramienta al modelo

In [80]:
modelo_con_herramienta = GROQ_LLM.bind_tools([multiply])

se vincula la herramienta multiply al modelo utilizando bind_tools. Esto permite que el modelo LLM llame a la función multiply cuando sea necesario

### Creación del grafo de mensajes

In [81]:
grafo = MessageGraph()

Se crea una instancia de MessageGraph 

In [82]:
grafo.add_node("oracle", modelo_con_herramienta)

Se añade un nodo llamado "oracle" que utiliza el modelo modelo_con_herramienta. Este nodo será el punto de entrada para el grafo

### Añadir un nodo de herramienta al grafo

In [83]:
herramienta_nodo = ToolNode([multiply])

In [84]:
grafo.add_node("multiply", herramienta_nodo)

Se crea un ToolNode con la herramienta multiply y se añade al grafo como un nodo llamado "multiply". Este nodo se encargará de ejecutar la función multiply cuando sea necesario

### Añadir un borde (edge) al grafo

In [85]:
grafo.add_edge("multiply", END)

Este borde conecta el nodo "multiply" con el nodo final END, indicando que después de procesar el nodo "multiply", el flujo del grafo terminará​

### Establecer el punto de entrada

In [86]:
grafo.set_entry_point("oracle")

Se establece "oracle" como el punto de entrada del grafo. Esto significa que el procesamiento del grafo comenzará desde este nodo

### Definir una función de enrutamiento condicional

In [87]:
def router(state: List[BaseMessage]) -> Literal["multiply", "__end__"]:
    tool_calls = state[-1].additional_kwargs.get("tool_calls", [])
    if len(tool_calls):
        return "multiply"
    else:
        return "__end__"

In [88]:
grafo.add_conditional_edges("oracle", router)

Se define una función router que determina el siguiente nodo a llamar basado en el estado actual. Si el último mensaje contiene llamadas a herramientas (tool_calls), se enruta al nodo "multiply". De lo contrario, se enruta al nodo __end__

### Compilar y ejecutar el grafo

In [89]:
runnable = grafo.compile()

In [90]:
resultado = runnable.invoke(HumanMessage("Cuanto es 123 * 456?"))
resultado

[HumanMessage(content='Cuanto es 123 * 456?', id='bb121aaf-06c0-4845-bb82-90c6b6eb62ef'),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_hc5c', 'function': {'arguments': '{"first_number":123,"second_number":456}', 'name': 'multiply'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_time': 0.243, 'completion_tokens': 76, 'prompt_time': 0.294, 'prompt_tokens': 917, 'queue_time': None, 'total_time': 0.5369999999999999, 'total_tokens': 993}, 'model_name': 'llama3-70b-8192', 'system_fingerprint': 'fp_abd29e8833', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-3123d9fd-b437-4c7a-b2da-5fcb0b60073b-0', tool_calls=[{'name': 'multiply', 'args': {'first_number': 123, 'second_number': 456}, 'id': 'call_hc5c'}]),
 ToolMessage(content='56088', name='multiply', id='4a3b036d-057f-4246-97c9-007328d75a38', tool_call_id='call_hc5c')]

In [91]:
print(resultado[2].content)

56088


Se compila el grafo en un objeto runnable y se invoca con el mensaje humano "Cuanto es 123 * 456?". Esto ejecuta el flujo definido en el grafo, pasando por el nodo "oracle" y luego, si es necesario, por el nodo "multiply" para calcular el resultado de la multiplicación

In [92]:
resultado = runnable.invoke(HumanMessage("¿Cual es tu nombre?"))
resultado

[HumanMessage(content='¿Cual es tu nombre?', id='622c5d8d-068d-47ab-aee9-32d09612c5e9'),
 AIMessage(content='Soy una inteligencia artificial, no tengo un nombre personal, pero puedo interactuar contigo de manera amistosa y ayudarte en lo que necesites. ¿En qué puedo ayudarte hoy?', response_metadata={'token_usage': {'completion_time': 0.136, 'completion_tokens': 41, 'prompt_time': 0.3, 'prompt_tokens': 915, 'queue_time': None, 'total_time': 0.436, 'total_tokens': 956}, 'model_name': 'llama3-70b-8192', 'system_fingerprint': 'fp_9625f32860', 'finish_reason': 'stop', 'logprobs': None}, id='run-c38c1141-ed86-44f6-a593-0993ad6fcdbd-0')]

In [93]:
print(resultado[1].content)

Soy una inteligencia artificial, no tengo un nombre personal, pero puedo interactuar contigo de manera amistosa y ayudarte en lo que necesites. ¿En qué puedo ayudarte hoy?


Esta consulta no necesita llamar a la herramienta y responde normalmente.